In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00


In [ ]:
import wandb
import numpy as np
from types import SimpleNamespace
import random

In [ ]:
wandb.login(key='type_your_api_key')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# You need to define a config file in the form of dictionary or yaml
sweep_config = {
    'method': 'grid',
    'name' : 'sweep cross entropy',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'epochs': {
            'values': [5,10]
        },
         'hidden_size':{
            'values':[32,64,128]
        },
        'activation': {
            'values': ['sigmoid','relu']
        },
        'loss': {
            'values': ['cross_entropy']
        },


    }
}

sweep_id = wandb.sweep(sweep=sweep_config, project='DL_TA(Ass_1)')

Create sweep with ID: 2f9yewzu
Sweep URL: https://wandb.ai/cs22s015/DL_TA%28Ass_1%29/sweeps/2f9yewzu


In [ ]:
# Load MNIST data using Keras
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
x_train,x_val, y_train, y_val=train_test_split(x_train,y_train, test_size=0.2,shuffle=True,random_state=42)
print(x_train.shape)

# Preprocess the data
x_train = x_train.astype('float32') / 255.0
x_val = x_val.astype('float32') / 255.0

# Flatten the images
x_train = x_train.reshape((-1, 28 * 28))
print(x_train.shape)
x_val = x_val.reshape((-1, 28 * 28))

# Convert to PyTorch tensors
x_train = torch.from_numpy(x_train)
y_train = torch.from_numpy(y_train)
x_val = torch.from_numpy(x_val)
y_val = torch.from_numpy(y_val)

(60000, 28, 28)
(48000, 28, 28)
(48000, 784)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
import numpy as np

# Define a simple 2-layer neural network
class SimpleNN(nn.Module):
    def __init__(self, input_size, output_size, config):
        super(SimpleNN, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = config['hidden_size']
        self.activation = config['activation']
        self.config = config
        # Define the first fully connected layer
        self.fc1 = nn.Linear(self.input_size, self.hidden_size)
        # Choose activation function based on the configuration
        if self.activation == 'sigmoid':
            self.activation = nn.Sigmoid()
        elif self.activation == 'relu':
            self.activation = nn.ReLU()
        # Define the second fully connected layer
        self.fc2 = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, x):
        # Forward pass through the network
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x

    def train(self, model, criterion, optimizer, x_train, y_train, x_val, y_val):
        # Training loop
        num_epochs = self.config['epochs']
        total_train_acc = []
        total_loss = []
        for epoch in range(num_epochs):
            inputs = x_train
            labels = y_train

            # Forward pass
            outputs = model(inputs)

            # Compute the loss
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()

            # Print the loss and accuracy for every epoch
            pred = torch.argmax(outputs, dim=1)
            tr_accuracy = torch.sum(pred == labels).item() / len(labels)
            wandb.log({'train loss': loss.item(), 'train accuracy': tr_accuracy * 100, 'epoch': epoch+1})

            # Evaluate the model on the validation set
            with torch.no_grad():
                outputs = model(x_val)
                predicted_labels = torch.argmax(outputs, dim=1)
                val_accuracy = torch.sum(predicted_labels == y_val).item() / len(y_val)
                wandb.log({'val_accuracy': val_accuracy * 100})

            # Update weights
            optimizer.step()


In [ ]:
def main():
    '''
    WandB calls main function each time with differnet combination.

    We can retrive the same and use the same values for our hypermeters.

    '''


    with wandb.init() as run:

        run_name="-ac_"+wandb.config.activation+"-hs"+str(wandb.config.hidden_size)
        wandb.run.name=run_name
        #obj=NN(wandb.config['num_layers'],wandb.config['hidden_size'])

        model = SimpleNN(784,10,wandb.config)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.01,betas=(0.9, 0.999))
        model.train( model,criterion,optimizer,x_train,y_train,x_val,y_val)


wandb.agent(sweep_id, function=main,count=10) # calls main function for count number of times.
wandb.finish()

wandb: Agent Starting Run: q9vhjsul with config:
wandb: 	activation: sigmoid
wandb: 	epochs: 5
wandb: 	hidden_size: 32
wandb: 	loss: cross_entropy


epoch,▁▃▅▆█
train accuracy,▁▅▇██
train loss,█▆▄▂▁
val_accuracy,▁▅▇▇█
epoch,5
train accuracy,58.43333
train loss,1.87821
val_accuracy,58.40833


wandb: Agent Starting Run: otq5p3ld with config:
wandb: 	activation: sigmoid
wandb: 	epochs: 5
wandb: 	hidden_size: 64
wandb: 	loss: cross_entropy


epoch,▁▃▅▆█
train accuracy,▁▃▆▇█
train loss,█▆▄▃▁
val_accuracy,▁▃▆▇█
epoch,5
train accuracy,64.88542
train loss,1.71328
val_accuracy,64.75


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 8zev6kdt with config:
wandb: 	activation: sigmoid
wandb: 	epochs: 5
wandb: 	hidden_size: 128
wandb: 	loss: cross_entropy


...

epoch,▁▃▅▆█
train accuracy,▁▃▇▇█
train loss,█▆▄▃▁
val_accuracy,▁▃▇▇█
epoch,5
train accuracy,77.94792
train loss,1.04831
val_accuracy,78.3


wandb: Agent Starting Run: 4j677hld with config:
wandb: 	activation: relu
wandb: 	epochs: 5
wandb: 	hidden_size: 128
wandb: 	loss: cross_entropy


epoch,▁▃▅▆█
train accuracy,▁▆▇██
train loss,█▆▄▂▁
val_accuracy,▁▆▇██
epoch,5
train accuracy,81.42917
train loss,0.72648
val_accuracy,81.05833


wandb: Agent Starting Run: ofne6qrm with config:
wandb: 	activation: relu
wandb: 	epochs: 10
wandb: 	hidden_size: 32
wandb: 	loss: cross_entropy


epoch,▁▂▃▃▄▅▆▆▇█
train accuracy,▁▃▅▆▆▇████
train loss,█▇▆▅▄▃▂▂▁▁
val_accuracy,▁▃▅▆▆▇████
epoch,10
train accuracy,83.53125
train loss,0.61643
val_accuracy,83.325


#References
[wandb tutorial](https://theaisummer.com/weights-and-biases-tutorial/)